In [ ]:
import json, yaml, os, time
import pandas as pd
from searchtweets import load_credentials, gen_rule_payload, ResultStream
from tweet_parser.tweet import Tweet
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

query = '"Joe Mixon" "Foot" from:InsideInjuries OR from:ProFootballDoc OR from:AdamSchefter'

results_per_call = 100  # 100 for sandbox, 500 for paid tiers
to_date = '2020-11-15' # format YYYY-MM-DD HH:MM (hour and minutes optional)
from_date = '2020-11-1'  # format YYYY-MM-DD HH:MM (hour and minutes optional)

# max number of tweets to collect
max_results = 5

# Script prints an update to the CLI every time it collected another X Tweets
print_after = 5

# opening the communication
premium_search_args = load_credentials("twitter_keys.yaml",yaml_key="search_tweets_api", env_overwrite=False)

# creating the payload for search
rule = gen_rule_payload(query,results_per_call=results_per_call,from_date=from_date,to_date=to_date)

#returning the results from the payload and stops at max_results
rs = ResultStream(rule_payload=rule,max_results=max_results,**premium_search_args)

# working properly, need to filter out retweets by using retweeted_status to remove retweets would be easiest in a dataframe
# need to format it for a pandas DF and write into csv instead of writing the full dictionary
# find headers within the data so that the differing size rows get split up properly
final_df = pd.DataFrame()

# loop to open FILENAME as f and write to f the value of tweet which would be results_per_call
for tweet in rs.stream():
    df = pd.json_normalize(tweet)
    final_df = pd.concat([final_df, df])
final_df = final_df[['user.name','user.screen_name','quote_count','retweet_count','retweeted','text','extended_tweet.full_text','retweeted_status.user.screen_name','retweeted_status.text','quoted_status.user.screen_name','quoted_status.text']]
#final_df.to_csv('injury_tweets.csv',index=False)
print('done')
final_df.head